# Imports


In [16]:
import numpy as np
import gym

# Create environment

There is 500 states :
* 5x5 grid
* 5 positions for the client (R,Y,G,B, in cab)
* 4 drop positions

And 6 actions :
* south, north, east, west
* pick / drop


In [10]:
env = gym.make("Taxi-v3").env

env.reset()
env.render()
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
print("Current State {}".format(env.s))

state = env.encode(3,1,0,3)
env.s = state
env.render()
print("Current State {}".format(env.s))

print("reward table {}".format(env.P[state]))


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)
Current State 174
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Current State 323
reward table {0: [(1.0, 423, -1, False)], 1: [(1.0, 223, -1, False)], 2: [(1.0, 343, -1, False)], 3: [(1.0, 323, -1, False)], 4: [(1.0, 323, -10, False)], 5: [(1.0, 323, -10, False)]}


# random walk through

In [111]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1796
Penalties incurred: 586


In [115]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, maxframe=50):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        if i >= maxframe:
            print(' ---> playback stopped after {} frames'.format(maxframe))
            break
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 51
State: 388
Action: 5
Reward: -10
 ---> playback stopped after 50 frames


# Train using QTable

### Train the model

In [88]:
import random

qtable = np.zeros((env.observation_space.n, env.action_space.n))

epoch_count = 100000
epsilon = 0.1
alpha = 0.1
gamma = 0.6

for epoch in range(epoch_count):
    env.reset()
    done = False
    
    state = env.s
    
    while not done:
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(qtable[state,:]) # Exploit learned values

        next_state, reward, done, info = env.step(action)
        
        old_value = qtable[state, action]
        next_max = np.max(qtable[next_state,:])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        qtable[state, action] = new_value
        
        state = next_state
        
    if epoch % 1000 == 0:
        clear_output(wait=True)
        print (epoch)

print ("trained")


99000
trained


### Test the model

In [131]:
env.reset()
state = env.s

steps = 0
done = False
penalties = 0

frames = []

while not done:
    action = np.argmax(qtable[state]) # Exploit learned values
    state, reward, done, info = env.step(action)

    steps += 1
    if reward == -10:
        penalties += 1
        
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
        
    if steps > 25:
        done = True
        
print("Timesteps taken: {}".format(steps))
print("Penalties incurred: {}".format(penalties))


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def draw(x):
    clear_output(wait=True)
    frame = frames[x]
    print(frame['frame'])
    print(f"Timestep: {x + 1}")
    print(f"State: {frame['state']}")
    print(f"Action: {frame['action']}")
    print(f"Reward: {frame['reward']}")
          
interact(draw, x=widgets.IntSlider(min=0, max=len(frames)-1, step=1, value=0));

Timesteps taken: 14
Penalties incurred: 0


interactive(children=(IntSlider(value=0, description='x', max=13), Output()), _dom_classes=('widget-interact',…